In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sb

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import HDBSCAN
from kmodes.kmodes import KModes

Loading data (loads binary dataframe, for using new version with both binary and age data just take only the binary)

In [ ]:
df = pd.read_csv('SDGEproperty_upgrade_data.csv')
df.info()

The first section prepares the binary data for clustering. This includes: pulling only desired columns (including selecting properties based on certain conditions) and then selecting only properties with >1 upgrade.

In [ ]:
# For pulling specific data

columns = list(df)[11:39]
column_set = [
    'Solar PV',
    'Battery storage',
    'Electric Vehicle Charger',
    'Electrical Panel Upgrade',
    'Water Heater',
    'Cool Roof',
    'Kitchen Remodel',
    'Bathroom Remodel',
    'AC',
    'Reroof',
    'Spa/Pool'
    ]

# df_u = df[df['Cool Roof'] == 1]
# df_u = df_u[df_u['Electrical Panel Upgrade'] == 0]
# df_u = df[(df['AC'] == 0) & (df['Cool Roof'] == 1)]
df_u = df[columns]
# df_u = df_u[df_u[column_set[8]] == 1]
# data = df_u.to_numpy(dtype='int')
# dsc = data[~np.all(data == 0, axis=1)]
df_u.head()

In [ ]:
print(column_set)

Brief intermission to check conditional probability for an upgrade:

In [45]:
# takes dataframe, an upgrade to check for (x) and the upgrade set (Y)
# to give result of P(x|y_i) for y_i in Y
def cond_probs(df, upgrade, u_set):
    total = len(df)
    p_A = float(len(df[df[upgrade] == 1])/total)
    print("Normal probability: " + str(p_A))
    for u in u_set:
        p_B = float(len(df[df[u] == 1])/total)
        p_U = float(len(df[(df[upgrade] == 1) & (df[u] == 1)])/total)
        p_C = float(p_U/p_B)
        print("Conditional probability given " + u + ": " + str(p_C))

In [ ]:
cond_probs(df_u, 'Cool Roof', column_set)

Back to preparing data (checking for >1 upgrade)

In [5]:
def check_mult(bool_list):
    mult = 0
    for val in bool_list:
        if val == 1:
            mult += 1
    if mult >= 1:
        return True
    else:
        return False

In [ ]:
i = 1
bools = []
for row in df_u.itertuples():
    bool_list = list(row[1:])
    if check_mult(bool_list):
        bools.append(bool_list)
    if i % 50000 == 0:
        print(i)
    i += 1

Finally, property data is set as binary numpy array

In [ ]:
data = np.array(bools)
train, test = train_test_split(data, test_size = 0.2)
sub, sub2 = train_test_split(test, test_size = 0.5)
print(len(train))
print(len(test))
print(len(sub))
print(data[0:5])

In [ ]:
# checks percentage of full data that is being clustered (based on conditions)
print((float(len(data)/len(df_u))) * 100)

Correlation, Cosine Similarity

In [ ]:
correlation = np.corrcoef(data.T)
heatmap = sb.heatmap(correlation, xticklabels=column_set, annot=False)
plt.show()

In [ ]:
similarities = cosine_similarity(data.T)
heatmap = sb.heatmap(similarities, yticklabels=columns, xticklabels=columns, vmax=0.5, annot=False)
plt.show()

The next section is for K-modes clustering. This essentially works the same as k-means with the difference being how cost is calculated. Instead of calculating cost as euclidean distance from cluster centroids to data points, cost is calculated as the number of differences in values between centroids and data. As a result, k-modes is more effective for binary data (like the binary upgrade data). The first section takes a smaller set of the overall data set to test with many values of k so an educated decision can be made for k.

In [ ]:
# calculates cost for different k

cost = []
centroids = []
labels = []
for i in range(2, 16):
    print(i)
    km_h = KModes(n_clusters=i, init='Huang', n_init = 1)
    km_h.fit_predict(sub)
    cost.append(km_h.cost_)
    centroids.append(km_h.cluster_centroids_)
    labels.append(km_h.labels_)
print(len(cost))

In [ ]:
# cost vs. k (use elbow method as a rough way to decide on good k)

x = np.arange(2, 16, 1)
plt.plot(x, cost)

Now, clustering is conducted with the entire data set

In [ ]:
# clustering, prints final cluster centroids when finished
# n_init sets how many runs to do, feel free to change

km_huang = KModes(n_clusters=10, init = "Huang", n_init = 5, verbose=1)
km_huang.fit_predict(data)
print(len(km_huang.labels_))
print(km_huang.cluster_centroids_)

In [ ]:
# prints number of properties in each cluster

totals_huang = {}
for i in range(0, len(km_huang.labels_)):
    label_huang = km_huang.labels_[i]
    if label_huang in totals_huang.keys():
        totals_huang[label_huang] += 1
    else:
        totals_huang[label_huang] = 1

for i in range(0, len(totals_huang.keys())):
    print(i, totals_huang[i])

Bernoulli Mixture Method (BMM) section. BMM, in the simplest terms, clusters the data into clusters which are built from multiple bernoulli distributions. The result is you have k clusters, each with their own cluster likelihood (how likely a property is to fall into that cluster), and in each cluster the likelihood of a certain upgrade being present (results of the bernoulli distributions, the BERNOULLI MIXTURE).

In [54]:
# import statements specific to BMM since there are a few
from bayespy.nodes import Categorical, Dirichlet
from bayespy.nodes import Beta
from bayespy.nodes import Mixture, Bernoulli
from bayespy.inference import VB
import bayespy.plot as bpplt

In [64]:
# setup for BMM

bern = data.copy()
N = len(bern)
D = 11
K = 1 # pick multiple and check results, not a perfect science

R = Dirichlet(K*[1e-5], name='R')               # sets initial properties for each cluster
Z = Categorical(R, plates=(N,1), name='Z')      
P = Beta([0.5, 0.5], plates=(D,K), name='P')    # sets initial properties for each upgrade in each cluster
X = Mixture(Z, Bernoulli, P)

In [ ]:
# inference
Q = VB(Z, R, X, P)
P.initialize_from_random()
X.observe(bern)
Q.update(repeat=1000)

The last section makes hinton plots for the BMM. Alas, the libary's function does not allow for labels. To read the plots, use the boxes-a bigger box means more likely. Clusters go left to right, upgrades go top to bottom.

In [ ]:
# Cluster P
bpplt.hinton(R)

In [ ]:
# Upgrade P per cluster
bpplt.hinton(P)